# Program optimization

So, you have a program, which implements an algorithm. It works. Results make sense. You have a suite of tests with reasonable coverage. 

The only *slight* problem is your program is slow. Sloth slow. You want to improve performance, i.e. optimize the program.


**Fact:** Whenever you try to guess what is slow in your program, ** you guess wrong. **

## The optimization loop

1. Profile the program. Identify the hotspots.

2. Optimize the bottleneck. *Only* touch the bottleneck, nothing else.

3. Verify that the program still works (**tests!**)

4. Measure the execution time. 

5. Goto 1.

## Profiling

e.g. GNU `gprof`, `ccachegrind`, `google-perftools`, `yep`, ...

In Python, use the `cProfile` module from the standard library for the function-level profiling. 
In Jupyter, there's a convenince magic, `%prun`.

As an example, we consider the adaptive integration scheme from a couple of weeks back.

In [1]:
import numpy as np

In [2]:
# A rectangle is (start, width)
# A list element is (-weight, (start, width))

def make_rect(a, b, f):
    """Make a rectangle for the interval [a, b]"""
    rect = (a, b-a)
    xm = a + rect[1] / 2.
    return (-f(xm) * rect[1], rect)
    

def get_max_elem(lst, key=None):
    """Find and remove the maximum element from the list.
    
    Find the max element (according to the parameter `key`,
    which is a callable), remove it from the list, and
    return both the element and the rest.
    """
    if key is None:
        # use the identity function
        key = lambda x: x
    
    # find the max element
    elem = max(lst, key=key)
    
    # find its position in the list
    idx = lst.index(elem)
    
    return elem, lst[:idx] + lst[idx+1:]

    
def adapt_rect_list(f, a, b, npts):
    """Integrate f(x) from a to b using npts steps.
    
    Uses an adaptive algorithm.
    """
    lst = []

    # start from a single rectangle
    item = make_rect(a, b, f)
    lst.append(item)
    
    # loop
    for _ in range(npts):
        # get the largest one
        rect, lst = get_max_elem(lst, lambda x: -x[0])
        w, (start, width) = rect
                
        # and split it into two halves
        c = start + width / 2.
        
        rect1 = make_rect(start, c, f)
        rect2 = make_rect(c, start + width, f)
        
        lst.append(rect1)
        lst.append(rect2)
        
    # collect the answer
    return -sum(w for w, r in lst), lst

In [3]:
def func(x):
    return np.tan(x) / (1.0 + np.exp(x))

a, b = 0, 10

In [4]:
# this opens a profile in the Jupyter's pager

%prun adapt_rect_list(func, a, b, npts=1000)

In [5]:
# alternatively, save the `pstats.Stats` object and manipulate it 

stats = %prun -r -q adapt_rect_list(func, a, b, npts=5000)

stats.print_stats(20)

          12552510 function calls in 3.184 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     5000    1.849    0.000    2.988    0.001 {built-in method builtins.max}
 12502500    1.139    0.000    1.139    0.000 <ipython-input-2-d80a194c1915>:45(<lambda>)
     5000    0.114    0.000    3.106    0.001 <ipython-input-2-d80a194c1915>:11(get_max_elem)
        1    0.038    0.038    3.184    3.184 <ipython-input-2-d80a194c1915>:31(adapt_rect_list)
    10001    0.027    0.000    0.027    0.000 <ipython-input-3-0da83198508c>:1(func)
    10001    0.008    0.000    0.035    0.000 <ipython-input-2-d80a194c1915>:4(make_rect)
    10001    0.004    0.000    0.004    0.000 {method 'append' of 'list' objects}
     5000    0.004    0.000    0.004    0.000 {method 'index' of 'list' objects}
        1    0.001    0.001    0.001    0.001 {built-in method builtins.sum}
     5002    0.000    0.000    0.000    0.000 <ipython-input-2-d80a194c19

## Line level profiling

In python land, use the `line_profiler` module. It's not in the standard library, so we need to install it. Open the `Anaconda prompt`, then type `conda install line_profiler`.

If using `pip`, type `pip install line_profiler --user`.

There also is an associated Jupyter magic. For using it outside of Jupyter, consult the [documentation: https://github.com/rkern/line_profiler.](https://github.com/rkern/line_profiler)

In [8]:
!pip install line_profiler

  Cache entry deserialization failed, entry ignored
You are using pip version 9.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [9]:
%load_ext line_profiler

In [10]:
# note that we need to give it the name of the function to profile via a `-f` switch

%lprun -f adapt_rect_list adapt_rect_list(func, a, b, 1000)

In [11]:
# again, save the results object (via a `-r` switch)

lstats = %lprun -r -f adapt_rect_list adapt_rect_list(func, a, b, 1000)

lstats.print_stats()

Timer unit: 1e-06 s

Total time: 0.362079 s
File: <ipython-input-2-f0e5fa43cac1>
Function: adapt_rect_list at line 31

Line #      Hits         Time  Per Hit   % Time  Line Contents
    31                                           def adapt_rect_list(f, a, b, npts):
    32                                               """Integrate f(x) from a to b using npts steps.
    33                                               
    34                                               Uses an adaptive algorithm.
    35                                               """
    36         1          3.0      3.0      0.0      lst = []
    37                                           
    38                                               # start from a single rectangle
    39         1         44.0     44.0      0.0      item = make_rect(a, b, f)
    40         1          2.0      2.0      0.0      lst.append(item)
    41                                               
    42                                  

In [12]:
# since we spend 90% of time in the `get_max_elem` function, profile it

lstats = %lprun -r -f get_max_elem adapt_rect_list(func, a, b, 1000)

lstats.print_stats()

Timer unit: 1e-06 s

Total time: 0.332588 s
File: <ipython-input-2-f0e5fa43cac1>
Function: get_max_elem at line 11

Line #      Hits         Time  Per Hit   % Time  Line Contents
    11                                           def get_max_elem(lst, key=None):
    12                                               """Find and remove the maximum element from the list.
    13                                               
    14                                               Find the max element (according to the parameter `key`,
    15                                               which is a callable), remove it from the list, and
    16                                               return both the element and the rest.
    17                                               """
    18      1000       1364.0      1.4      0.4      if key is None:
    19                                                   # use the identity function
    20                                                   key = 

## The best optimization is often an algorithmic one

We have already implemented an improved version of the adaptive integration, using a priority queue. Let's profile it.

In [9]:
import heapq

# rect is (start, width)
# queue element is (-weight, (start, width))

def make_rect(a, b, f):
    """Make a rectangle for the interval [a, b]."""
    rect = (a, b-a)
    xm = a + rect[1] / 2.
    return (-f(xm) * rect[1], rect)
    
    
def adapt_rect(f, a, b, npts):
    hp = []

    # start from a single rectangle
    item = make_rect(a, b, f)
    heapq.heappush(hp, item)
    
    # loop
    for _ in range(npts):
        # get the largest one
        w, (start, width) = heapq.heappop(hp)
        
        # and split it into two halves
        c = start + width / 2.
        
        rect1 = make_rect(start, c, f)
        rect2 = make_rect(c, start + width, f)
        
        heapq.heappush(hp, rect1)
        heapq.heappush(hp, rect2)
    
    # collect the answer
    return -sum(w for w, r in hp), hp

In [10]:
stats = %prun -q -r adapt_rect(func, a, b, 5000)
stats.print_stats(20)

          40010 function calls in 0.111 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10001    0.060    0.000    0.060    0.000 <ipython-input-2-617a020dccdd>:1(func)
    10001    0.018    0.000    0.078    0.000 <ipython-input-9-a3d338d819a3>:6(make_rect)
        1    0.017    0.017    0.110    0.110 <ipython-input-9-a3d338d819a3>:13(adapt_rect)
     5000    0.010    0.000    0.010    0.000 {built-in method _heapq.heappop}
    10001    0.004    0.000    0.004    0.000 {built-in method _heapq.heappush}
        1    0.001    0.001    0.001    0.001 {built-in method builtins.sum}
     5002    0.001    0.000    0.001    0.000 <ipython-input-9-a3d338d819a3>:35(<genexpr>)
        1    0.000    0.000    0.111    0.111 <string>:1(<module>)
        1    0.000    0.000    0.111    0.111 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [12]:
lstats = %lprun -r -f adapt_rect adapt_rect(func, a, b, 1000)
lstats.print_stats()

Timer unit: 1e-06 s

Total time: 0.048421 s
File: <ipython-input-10-a3d338d819a3>
Function: adapt_rect at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
    13                                           def adapt_rect(f, a, b, npts):
    14         1            3      3.0      0.0      hp = []
    15                                           
    16                                               # start from a single rectangle
    17         1           58     58.0      0.1      item = make_rect(a, b, f)
    18         1            6      6.0      0.0      heapq.heappush(hp, item)
    19                                               
    20                                               # loop
    21      1001         1633      1.6      3.4      for _ in range(npts):
    22                                                   # get the largest one
    23      1000         4732      4.7      9.8          w, (start, width) = heapq.heappop(hp)
    24                    

## Further reading

1. [cProfile] Doug Hellmann, *profile and pstats — Performance Analysis*, https://pymotw.com/3/profile/

2. [line_profiler] Robert Kern, *Line profiler documentation*, https://github.com/rkern/line_profiler

3. [yep] Fabian Pedregosa, *Profiling in Python*, http://www.slideshare.net/FabianPedregosa/profiling-in-python-54685566

Note however that `yep` is reported to not work under Windows. However I personally have had a very good experience using it (on Linux systems) when writing extension modules in C/C++.